Pro správné fungování notebooku je potřeba mít v Jupyteru mít povolenou/nainstalovanou [integraci s Matplotlib](https://github.com/matplotlib/ipympl) a v prostředí Python kernelu mít nainstalované balíčky:
- psycopg2
- pymongo
- pandas
- matplotlib

In [1]:
%matplotlib widget
import sys; sys.path.insert(0, '.')
import os

Některé součásti řešení zde nebudeme demonstrovat v celé délce, použijeme funkce pro zpracování vstupních dat nebo pro připojení k databázím, které máme předdefinované v našich knihovních souborech. Hlavní strukturu zde ale nastíníme, počínaje stáhnutím a zpracováním vstupních souborů.

## Stažení zdrojových souborů

In [2]:
scrape_dir = 'scraped/'
if not os.path.isdir(scrape_dir):
    os.mkdir(scrape_dir)
len(os.listdir(scrape_dir))

122

In [43]:
import csv
from datetime import date, datetime
import time

from dateutil.relativedelta import relativedelta
import requests

def scrape(base_url, output_dir, start_date, end_date):
    any_downloads = False
    for ordinal in range(start_date.toordinal(), end_date.toordinal()):
        url = base_url + date.fromordinal(ordinal).strftime('%d.%m.%Y')
        path = date.fromordinal(ordinal).strftime('%Y-%m-%d') + '.txt'
        filename = os.path.join(output_dir, path)
        if os.path.isfile(filename):
            continue
        any_downloads = True
        print("Requesting %s..." % url, end='')
        try:
            request = requests.get(url)
            if not request.text:
                print(' Empty!')
                continue
            print(' OK')
            with open(filename, 'w') as handle:
                handle.write(request.text)
            time.sleep(0.2)
        except Exception as ex:
            print(' %s' % ex)
            continue
    if not any_downloads:
        print("All files already present.")

start_date = datetime.today() - relativedelta(months=4)
end_date = datetime.today()
scrape(
    base_url='https://www.cnb.cz/cs/financni-trhy/devizovy-trh/kurzy-devizoveho-trhu/kurzy-devizoveho-trhu/denni_kurz.txt?date=',
    start_date=start_date,
    end_date=end_date,
    output_dir=scrape_dir,
)
print("Got %s input files" % len(os.listdir(scrape_dir)))
print()
with open(scrape_dir + '/' + os.listdir(scrape_dir)[0], 'r') as f:
    print(f.read())

All files already present.
Got 122 input files

09.10.2020 #196
země|měna|množství|kód|kurz
Austrálie|dolar|1|AUD|16,526
Brazílie|real|1|BRL|4,120
Bulharsko|lev|1|BGN|13,862
Čína|žen-min-pi|1|CNY|3,430
Dánsko|koruna|1|DKK|3,643
EMU|euro|1|EUR|27,110
Filipíny|peso|100|PHP|47,490
Hongkong|dolar|1|HKD|2,966
Chorvatsko|kuna|1|HRK|3,579
Indie|rupie|100|INR|31,449
Indonesie|rupie|1000|IDR|1,563
Island|koruna|100|ISK|16,652
Izrael|nový šekel|1|ILS|6,802
Japonsko|jen|100|JPY|21,694
Jižní Afrika|rand|1|ZAR|1,395
Kanada|dolar|1|CAD|17,443
Korejská republika|won|100|KRW|2,007
Maďarsko|forint|100|HUF|7,610
Malajsie|ringgit|1|MYR|5,554
Mexiko|peso|1|MXN|1,081
MMF|ZPČ|1|XDR|32,441
Norsko|koruna|1|NOK|2,496
Nový Zéland|dolar|1|NZD|15,212
Polsko|zlotý|1|PLN|6,065
Rumunsko|leu|1|RON|5,565
Rusko|rubl|100|RUB|29,811
Singapur|dolar|1|SGD|16,958
Švédsko|koruna|1|SEK|2,602
Švýcarsko|frank|1|CHF|25,162
Thajsko|baht|100|THB|74,009
Turecko|lira|1|TRY|2,908
USA|dolar|1|USD|22,983
Velká Británie|libra|1|GBP|29,7

In [4]:
from scrape import parse
for input_file in parse(scrape_dir):
    print(input_file)
    break

{'date': datetime.datetime(2020, 10, 9, 0, 0), 'currency': {'country': 'Austrálie', 'name': 'dolar', 'code': 'AUD'}, 'lotSize': '1', 'price': '16,526'}


Nyní máme stažené všechny textové/CSV vstupní soubory a zpracované v takovém formátu, že je můžeme přímo vložit do MongoDB bez dalšího zpracování. Do tohoto bodu se veškeré zpracování skládalo z načtení CSV souboru a přidání data ke každému řádku tak, se dá dále zpracovávat.

In [5]:
from db_connects import MONGO_DB_CURRENCIES, MONGO_DB_COL_CURRENCIES, connect_to_mongodb

client = connect_to_mongodb()
collection = client[MONGO_DB_CURRENCIES][MONGO_DB_COL_CURRENCIES]
collection.drop()

res = collection.insert_many(parse(scrape_dir))
print("Loaded %s records to MongoDB" % len(res.inserted_ids))

collection.find_one()

Loaded 4026 records to MongoDB


{'_id': ObjectId('5fccceb6462ef441d1243dc2'),
 'date': datetime.datetime(2020, 10, 9, 0, 0),
 'currency': {'country': 'Austrálie', 'name': 'dolar', 'code': 'AUD'},
 'lotSize': '1',
 'price': '16,526'}

Takto vypadají všechny záznamy v MongoDB. Nyní je převedeme to PostgreSQL, konkrétně do normalizovaného formátu ve dvou tabulkách, jedna tabulka měn a jedna tabulka kurzů.

- `Měna = Kód měny (string, primární klíč) x Název (string) x Země (string)`
- `Kurz = Den (date) x Kód měny (cizí klíč) x Normalizovaný kurz (Float)`

(Float sice není ideální reprezentace pro finanční výpočty, ale pro naše účely postačuje.)

In [16]:
import json
from bson.json_util import dumps
from psycopg2 import extensions
from db_connects import connect_to_postgres

conn = connect_to_postgres()

conn.set_isolation_level(extensions.ISOLATION_LEVEL_AUTOCOMMIT)

cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS kurz")
cursor.execute("DROP TABLE IF EXISTS mena")
cursor.execute("CREATE TABLE mena (zeme varchar(100), nazev varchar(100), kod varchar(10) primary KEY)")
cursor.execute(
    "CREATE TABLE kurz (den DATE, kod varchar(10), "
    "CONSTRAINT fk_mena FOREIGN KEY(kod) REFERENCES mena(kod) ON DELETE SET NULL, "
    "normalizovany_kurz FLOAT)"
)

In [17]:
mena_res = collection.find({}, {"currency": 1, "_id": 0}).distinct("currency")
for mena_item in mena_res:
    cursor.execute("INSERT INTO mena VALUES ('{}', '{}', '{}')".format(
        mena_item["country"],
        mena_item["name"],
        mena_item["code"]
    ))

for item in collection.find({}, {"_id": 0}):
    cursor.execute("INSERT INTO kurz VALUES ('{}', '{}', '{}')".format(
        item["date"].strftime("%Y-%m-%d"),
        item["currency"]["code"],
        float(item["price"].replace(',', '.')) / int(item["lotSize"])
    ))

In [18]:
cursor.execute("SELECT * from mena")
print("%s rows" % cursor.rowcount)
for row in cursor:
    print(row)
    break
print()
cursor.execute("SELECT * from kurz")
print("%s rows" % cursor.rowcount)
for row in cursor:
    print(row)
    break

33 rows
('Austrálie', 'dolar', 'AUD')

4026 rows
(datetime.date(2020, 10, 9), 'AUD', 16.526)


Nyní máme všechna data ve strukturované reprezentaci v PostgreSQL a můžeme se pustit do jednotlivých úkolů.

## Úkol A

První úkol, který jsme si ze zadání vybrali, je vytvoření žebříčku měn, které v daném období nejvíce posílily/oslabily.

In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

cursor.execute(
    "select kod, normalizovany_kurz from kurz where den = (SELECT MIN(den) from kurz)"
    " ORDER BY kod ASC"
)
min_hash = dict(cursor)
cursor.execute(
    "select kod, normalizovany_kurz from kurz where den = (SELECT MAX(den) from kurz)"
    " GROUP BY kod, normalizovany_kurz ORDER BY kod ASC"
)
diff = {}
for item in cursor:
    diff[item[0]] = min_hash[item[0]] - item[1]
diff = {k: v for k, v in sorted(diff.items(), key=lambda x: -x[1])}

fig = plt.figure()
x = np.arange(len(diff))
plt.bar(x, height=diff.values())
plt.xticks(x, diff.keys(), rotation=-90);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [54]:
print("Between %s and %s the best performing currency was %s which changed by %s units." % (
    start_date.date(), end_date.date(), list(diff.items())[0][0], round(list(diff.items())[0][1], 2)
))

Between 2020-08-06 and 2020-12-06 the best performing currency was USD which changed by 0.31 units.
